In [1]:
import pandas as pd 
import numpy as np
import statsmodels.api as sm
from linearmodels import PooledOLS, PanelOLS,RandomEffects
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.stattools import adfuller
import scipy.stats as stats
from linearmodels.panel import compare
from linearmodels import PanelOLS, RandomEffects

d:\Software_installation\Anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
prediction_results = pd.read_csv(r"E:\毕业论文\数据\股吧数据\prediction_results.csv")
zssh = pd.read_csv(r"E:\毕业论文\数据\股吧数据\zssh000016.csv")

In [5]:
merge_df = pd.merge(left=zssh,right=prediction_results,how='inner',on='标题')
merge_df = merge_df[['阅读数','评论数','标题','发表时间','情感分类']]
merge_df.head(5)

,阅读数,评论数,标题,发表时间,情感分类
0,223,1,想要交易50etf期权在哪里交易？50ETF期权交易要注意什,2022-10-28,1
1,142,0,今天不是跌晕了就是涨晕了[拜神][拜神][拜神],2022-10-28,0
2,119,0,百手空单富追马云，坚决88倍杆杆做空上证50,2022-10-28,0
3,998,3,指数还在选择方向,2022-10-28,1
4,329,2,上证50遇到2300再度选择方向，直接回踩2050到月线上涨通道下轨或反弹到到短,2022-10-28,1


In [ ]:
merge_df = merge_df[['阅读数','评论数','标题','发表时间','情感分类']]

In [ ]:
merge_df.head(5)

In [ ]:
merge_df['情感分类'] = pd.Categorical(merge_df['情感分类'])
dummies = pd.get_dummies(merge_df['情感分类'], prefix='情感分类')

# 将哑变量合并回原数据框
df = pd.concat([merge_df, dummies], axis=1)

# 按照发表时间分组并汇总数据
result = df.groupby('发表时间').agg({
    '阅读数': 'sum',
    '评论数': 'sum',
    '标题':'count',
    '情感分类_0': 'sum',  # 情感分类0的总数
    '情感分类_1': 'sum',  # 情感分类1的总数
    '情感分类_2': 'sum'   # 情感分类2的总数
}).reset_index()

# 重命名列以匹配原始数据集
result.columns = ['Date', '总阅读数', '总评论数','Tpostnum','Negpostnum', '中性', 'Pospostnum']

# 添加代码列
result['Scode'] = 'zssh000016'

In [ ]:
result['Senti'] = (result['Pospostnum'] - result['Negpostnum']) / (result['Pospostnum'] + result['Negpostnum'])
result['MsgSenti'] = np.log((1+result['Pospostnum'])/(1+result['Negpostnum']))
result['Concern'] = np.log(1+result['Tpostnum'])
result['Agree'] = 1 - np.sqrt(1-result['Senti']**2)

In [ ]:
sz_return = pd.read_excel(r"E:\毕业论文\数据\股吧数据\上证50日回报率.xlsx")

In [ ]:
sz_return = sz_return[['Idxtrd01','Idxtrd08']]
sz_return.rename(columns={'Idxtrd01':'Date','Idxtrd08':'return'},inplace=True)

In [ ]:
sz_50 = sz_return.merge(result,how='left')

In [ ]:
sz_50.to_csv('sz_50_test.csv',encoding='utf-8-sig',index=None)

In [ ]:
prediction_results_50 = pd.read_csv(r"E:\毕业论文\数据\股吧数据\clean\prediction_50_all.csv")
sz_50_all = pd.read_csv(r"E:\毕业论文\数据\股吧数据\clean\50_all.csv_txt_clean.csv")

In [ ]:
prediction_results_50['情感分类'].value_counts()

In [ ]:
prediction_results_50 = prediction_results_50.rename(columns={'文本':'标题'})

In [ ]:
merge_df_all = pd.merge(left=sz_50_all,right=prediction_results_50,how='inner',on='标题')

In [ ]:
merge_df_all

In [ ]:
merge_df_all = merge_df_all.drop_duplicates(subset=['标题','情感分类'],ignore_index=True).reset_index()

In [ ]:
merge_df_all=merge_df_all.iloc[:,1:]

In [ ]:
merge_df_all

In [ ]:
merge_df_all['情感分类'] = pd.Categorical(merge_df_all['情感分类'])
dummies = pd.get_dummies(merge_df_all['情感分类'], prefix='情感分类')

# 将哑变量合并回原数据框
df_all = pd.concat([merge_df_all, dummies], axis=1)

# 按照发表时间分组并汇总数据
result_all = df_all.groupby(['发表时间','stock_code']).agg({
    '阅读数': 'sum',
    '评论数': 'sum',
    '标题':'count',
    '情感分类_0': 'sum',  # 情感分类0的总数
    '情感分类_1': 'sum',  # 情感分类1的总数
    '情感分类_2': 'sum'   # 情感分类2的总数
}).reset_index()

# 重命名列以匹配原始数据集
result_all.columns = ['Date','Scode','总阅读数', '总评论数','Tpostnum','Negpostnum', '中性', 'Pospostnum']


In [ ]:
result_all['Senti'] = (result_all['Pospostnum'] - result_all['Negpostnum']) / (result_all['Pospostnum'] + result_all['Negpostnum'])
result_all['MsgSenti'] = np.log((1+result_all['Pospostnum'])/(1+result_all['Negpostnum']))
result_all['Concern'] = np.log(1+result_all['Tpostnum'])
result_all['Agree'] = 1 - np.sqrt(1-result_all['Senti']**2)

In [ ]:
result_all

In [ ]:
result_all=result_all.fillna(value=0)

In [ ]:
result_all.to_csv('result_all_real.csv',index=False,encoding='utf_8_sig')

In [ ]:
import re
sz_huanshou = pd.read_excel('其他数据\日换手率.xlsx')
sz_trade = pd.read_excel('其他数据\日频股票数据.xlsx')
sz_return = pd.read_excel('其他数据\日收益率.xlsx')
sz_BM = pd.read_excel('其他数据\账面市值比季度数据.xlsx')
sz_EPS = pd.read_excel('其他数据\EPS季度数据.xlsx')
sz_PE_PB = pd.read_excel('其他数据\市盈率市净率.xlsx')
sz_uncertain = pd.read_excel("E:\毕业论文\数据\RERR_ChinaEPUIndexM.xlsx")
sz_roe = pd.read_excel(r"其他数据\roe.xlsx")

In [ ]:
sz_days = sz_return.merge(sz_huanshou, on=['Stkcd','Trddt'], how='left')
sz_days = sz_days.merge(sz_trade, on=['Stkcd','Trddt'], how='left')
sz_days = sz_days.merge(sz_PE_PB, on=['Stkcd','Trddt'], how='left')
sz_BM = sz_BM.merge(sz_EPS,how='left',on=['Accper','Stkcd'])
sz_season = sz_BM.merge(sz_roe,how='left',on=['Accper','Stkcd'])
sz_season.drop(columns=['ShortName_x','ShortName_y'],inplace=True)
sz_season.rename(columns={'Accper':'Trddt'},inplace=True)
sz_season.drop(columns=['ShortName'],inplace=True)
sz_control = sz_days.merge(sz_season,how='left',on=['Stkcd','Trddt'])
sz_control = sz_control.rename(columns={'Trddt':'Date'})
sz_control.to_csv()
sz_control['Month'] = pd.to_datetime(sz_control['Date']).dt.to_period('M').astype(str)
sz_uncertain['Date'] = sz_uncertain['Date'].astype(str)
merged_df = pd.merge(sz_control, sz_uncertain, left_on='Month', right_on='Date', how='left')
merged_df = merged_df.drop(columns=['Date_y','Month'])
merged_df.rename(columns={'Date_x':'Date'},inplace=True)
merged_df['B/M'] = merged_df['B/M'].interpolate()
merged_df['EPS'] = merged_df['EPS'].interpolate()
merged_df['Roe'] = merged_df['Roe'].interpolate()
merged_df[['B/M','EPS']] = merged_df[['B/M','EPS']].bfill()
merged_df.rename(columns={'Trddt':'Date'},inplace=True)
merged_df.rename(columns={'Stkcd':'Scode'},inplace=True)
merged_df.rename(columns={'index':'Uncertain'},inplace=True)
sz_result_all = merged_df.merge(result_all,how='left',on=['Scode','Date'])
sz_result_all

In [ ]:
sz_result_all=sz_result_all.rename(columns={'总阅读数':'Readnum','总评论数':'Commentnum','中性':'Neupostnum'})
sz_result_all.to_csv('E:\毕业论文\数据\东方财富网帖子\sz_all_real.csv',index=0)

In [ ]:
sz_all = pd.read_csv('sz_all_real.csv')
sz_all = sz_all.set_index('Date')